In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.metrics.pairwise import cosine_similarity
from typing import *
import copy
from deeppavlov.dataset_readers.ontonotes_reader import OntonotesReader

/home/kostrovskiy/prog/AI/NLP/DeepPavlov/env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
[nltk_data] Downloading package punkt to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
2018-09-06 18:09:42.610 DEBU

In [2]:
from src.fewshot_ner_viz_component.data_processing import *
from src.fewshot_ner_viz_component.utils import *
from src.fewshot_ner_viz_component.fewshot_ner_binary_classifier import *

In [3]:
dataset_orig = read_data()
ne_type = 'PERSON'
dataset = filter_dataset_by_ne_types(dataset_orig, ne_type)

Num of train sentences: 75187
Num of valid sentences: 9603
Num of test sentences: 9479
[(['Actions', 'had', 'to', 'be', 'taken', 'to', 'break', 'through', 'the', 'blockade', '.'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']), (['On', 'a', 'night', 'in', 'late', 'July', '1940', ',', 'the', 'atmosphere', 'in', 'Zhuanbi', 'Village', 'in', 'Shaanxi', 'was', 'unusual', '.'], ['O', 'B-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'O', 'O', 'O', 'O', 'B-GPE', 'I-GPE', 'O', 'B-GPE', 'O', 'O', 'O']), (['Villager', 'Xiao', 'Jianghe', 'has', 'a', 'vivid', 'memory', 'of', 'this', 'piece', 'of', 'history', '.'], ['O', 'B-PERSON', 'I-PERSON', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']), (['On', 'that', 'dark', 'night', ',', 'everyone', 'was', 'sleeping', 'when', 'human', 'voices', 'and', 'neighing', 'horses', 'were', 'heard', 'within', 'the', 'village', '.'], ['O', 'B-TIME', 'I-TIME', 'I-TIME', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

In [4]:
N_EXAMPLES = 10
N_TEST_SENTENCES = 100
N_SAMPLES = 10
MODELS = ['svm']
MODEL_PARAMS = {'ne_centroid': {'sim_type': 'cosine'},
               'ne_nearest': {'sim_type': 'cosine'},
               'weighted_kNN': {'k': 1, 'metric': 'cosine', 'use_class_weights': False, 'use_sim_weights': True},
               'centroid_kNN': {'k': 10, 'metric': 'dot_prod', 'use_class_weights': False},
               'svm': {}}
SVM_KERNELS = ['linear', 'rbf', 'sigmoid', 'cosine_similarity']

In [5]:
tf.reset_default_graph()
embedder = CompositeEmbedder(use_elmo=True, elmo_scale=1, cap_scale=1, use_cap_feat=False, use_glove=False)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.


2018-09-06 18:09:54.939 INFO in 'tensorflow'['tf_logging'] at line 160: Using /tmp/tfhub_modules to cache modules.


INFO:tensorflow:Initialize variable module/aggregation/scaling:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with aggregation/scaling


2018-09-06 18:09:55.364 INFO in 'tensorflow'['tf_logging'] at line 116: Initialize variable module/aggregation/scaling:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with aggregation/scaling


INFO:tensorflow:Initialize variable module/aggregation/weights:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with aggregation/weights


...

2018-09-06 18:09:55.749 INFO in 'tensorflow'['tf_logging'] at line 116: Initialize variable module/bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/kernel:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/kernel


INFO:tensorflow:Initialize variable module/bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/projection/kernel:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/projection/kernel


2018-09-06 18:09:55.756 INFO in 'tensorflow'['tf_logging'] at line 116: Initialize variable module/bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/projection/kernel:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/projection/kernel


INFO:tensorflow:Initialize variable module/bilm/char_embed:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/char_embed


2018-09-06 18:09:55.768 INFO in 'tensorflow'['tf_logging'] at line 116: Initialize variable module/bilm/char_embed:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/char_embed


### Make samples

In [6]:
data_samples = []
for i in range(N_SAMPLES):
    tokens_train,tags_train = get_data_sample(dataset['train'], N_EXAMPLES)
    tokens_test,tags_test = get_data_sample(dataset['test'], min(N_TEST_SENTENCES, len(dataset['test'])))
    y_test = np.array([1 if t == 'T' else 0 for seq in tags_test for t in seq])
    data_samples.append({'train': {'tokens': tokens_train, 'tags': tags_train}, 'test': {'tokens': tokens_test, 'tags': tags_test, 'y': y_test}})

### Prepare embedder params

In [7]:
f1_scores = {}
for i, svm_kernel in enumerate(SVM_KERNELS):
    print('============ {} ============'.format(svm_kernel))
    label = svm_kernel
    if svm_kernel == 'cosine_similarity':
        svm_kernel = cosine_similarity
    f1_scores[label] = {}
    for model in MODELS:
        f1_scores[label][model] = {'vals': [], 'avg': 0, 'std': 0}
    for j, data_sample in enumerate(data_samples):
        print('Sample {}/{}'.format(j+1, N_SAMPLES))
        tokens_train = data_sample['train']['tokens']
        tags_train = data_sample['train']['tags']
        tokens_test = data_sample['test']['tokens']
        tags_test = data_sample['test']['tags']
        y_test = data_sample['test']['y']
        ner_bin_clf = FewshotNerBinaryClassifier(embedder, svm_kernel=svm_kernel)
        ner_bin_clf.train_on_batch(tokens_train, tags_train)
        results = ner_bin_clf.predict(tokens_test, MODELS, MODEL_PARAMS)
        for model in MODELS:    
            pred = results[model]['pred']
            f1 = f1_score(y_test, pred)
            f1_scores[label][model]['vals'].append(f1)
            print('Model: {}, F1 (tokens) = {:.2f}'.format(model, f1*100))
    print('Average F1 scores:')
    for model in MODELS:
        f1_avg = np.mean(f1_scores[label][model]['vals'])
        f1_std = np.std(f1_scores[label][model]['vals'])
        f1_scores[label][model]['avg'] = f1_avg
        f1_scores[label][model]['std'] = f1_std
        print('Model: {}, avg F1 (tokens) = {:.2f} std = {:.2f}'.format(model, f1_avg*100, f1_std*100))

============ linear ============
Sample 1/10
Train
# ne: 22, # tokens: 238
n_samples: 238
SVM classifier model
Model: svm, F1 (tokens) = 78.48
Sample 2/10
Train
# ne: 17, # tokens: 202
n_samples: 202
SVM classifier model
Model: svm, F1 (tokens) = 88.57
Sample 3/10
Train
# ne: 16, # tokens: 282
n_samples: 282
SVM classifier model
Model: svm, F1 (tokens) = 81.46
Sample 4/10
Train
# ne: 20, # tokens: 164
n_samples: 164
SVM classifier model
Model: svm, F1 (tokens) = 88.20
Sample 5/10
Train
# ne: 23, # tokens: 258
n_samples: 258
SVM classifier model
Model: svm, F1 (tokens) = 82.08
Sample 6/10
Train
# ne: 21, # tokens: 233
n_samples: 233
SVM classifier model
Model: svm, F1 (tokens) = 90.24
Sample 7/10
Train
# ne: 17, # tokens: 195
n_samples: 195
SVM classifier model
Model: svm, F1 (tokens) = 80.63
Sample 8/10
Train
# ne: 18, # tokens: 280
n_samples: 280
SVM classifier model
Model: svm, F1 (tokens) = 85.71
Sample 9/10
Train
# ne: 18, # tokens: 246
n_samples: 246
SVM classifier model
Model: sv

KeyboardInterrupt: 

In [ ]:
print(f1_scores)

In [ ]:
def make_table_caption(labels):
    s = '{:<8s}'
    for lab in labels:
        s += '\t{:<8s}'
    return s.format('MODEL', *labels)

def make_table_line(model, scores):
    s = '{:<8s}'
    for score in scores:
        s += '\t{:<8.2f}'
    return s.format(model, *scores)

In [ ]:
labels = [kernel_name for kernel_name in SVM_KERNELS]
print(make_table_caption(labels))
for model in MODELS:
    print(make_table_line(model, [f1_scores[lab][model]['avg']*100 for lab in labels]))    